# 📘 Mô phỏng hệ thống hàng đợi M/M/c/c (Erlang-B)
Trong notebook này, chúng ta sẽ:
- Tính toán xác suất từ chối theo lý thuyết Erlang-B
- Mô phỏng hệ thống M/M/c/c bằng thư viện `simpy`
- So sánh kết quả mô phỏng với lý thuyết


In [18]:
# 📐 Hàm tính Erlang-B
import math

def erlang_b(c, A):
    B = (A ** c) / math.factorial(c)
    denom = sum((A ** k) / math.factorial(k) for k in range(c + 1))
    return B/(1+denom)

# Ví dụ: c = 3 servers, lambda = 5, mu = 2 => A = 2.5
c = 10
lambd = 15
mu = 3
A = lambd / mu
blocking_theory = erlang_b(c, A)
print(f"Xác suất từ chối (lý thuyết): {blocking_theory:.8f}")

Xác suất từ chối (lý thuyết): 0.01825983


In [19]:
# 🧪 Mô phỏng hệ thống M/M/c/c bằng simpy
import simpy
import random

RANDOM_SEED = 42
SIM_TIME = 10000

def customer(env, servers, mu, stats):
    with servers.request() as req:
        result = yield req | env.timeout(0)
        if req in result:
            service_time = random.expovariate(mu)
            yield env.timeout(service_time)
            stats['served'] += 1
        else:
            stats['rejected'] += 1

def generator(env, lambd, mu, c, stats):
    servers = simpy.Resource(env, capacity=c)
    while True:
        yield env.timeout(random.expovariate(lambd))
        env.process(customer(env, servers, mu, stats))

random.seed(RANDOM_SEED)
env = simpy.Environment()
stats = {'served': 0, 'rejected': 0}
env.process(generator(env, lambd, mu, c, stats))
env.run(until=SIM_TIME)

total = stats['served'] + stats['rejected']
blocking_sim = stats['rejected'] / total
print(f"Xác suất từ chối (mô phỏng): {blocking_sim:.8f}")

Xác suất từ chối (mô phỏng): 0.01802454


In [20]:
# 📊 So sánh lý thuyết và mô phỏng
print(f"Từ chối (Lý thuyết Erlang-B): {blocking_theory:.8f}")
print(f"Từ chối (Mô phỏng simpy): {blocking_sim:.8f}")

Từ chối (Lý thuyết Erlang-B): 0.01825983
Từ chối (Mô phỏng simpy): 0.01802454
